In [2]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import re
import gc
from sklearn.metrics import mean_squared_error, roc_auc_score

from xgboost import XGBClassifier, XGBRFRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from lightgbm import LGBMModel,LGBMClassifier
import copy

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-uuoun520 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
from sklearn.feature_selection import RFE

# clf = SVC(kernel='sigmoid', random_state=42, probability=True, C=0.1, class_weight='balanced')
clf = LogisticRegression(solver='sag', random_state=42, class_weight='balanced')
# clf = XGBClassifier(
#         random_state = 42,
#         objective='binary:logistic',
#         n_estimators=500,
#         learning_rate=1e-3,
#         max_depth=3,
#         subsample=0.7,
#         scale_pos_weight=np.unique(y_train, return_counts=True)[1][0] / np.unique(y_train, return_counts=True)[1][1]
#     )
    

val_evals = []
aucs = []
f1s = []
for fidx in range(5):
    X_train = np.load('/work/zg78/som_folds/X_train_{}.npy'.format(fidx))
    y_train = np.load('/work/zg78/som_folds/y_train_{}.npy'.format(fidx))
    X_val = np.load('/work/zg78/som_folds/X_val_{}.npy'.format(fidx))
    y_val = np.load('/work/zg78/som_folds/y_val_{}.npy'.format(fidx))
    
    print(X_train.shape, y_train.shape, np.unique(y_val, return_counts=True))
    
    std_scaler = StandardScaler()
    std_scaler.fit(X_train)
    X_train = std_scaler.transform(X_train)
    X_val = std_scaler.transform(X_val)
        
    clf.fit(X_train, y_train)
    
    train_acc = metrics.accuracy_score(y_train, clf.predict(X_train))
    val_acc = metrics.accuracy_score(y_val, clf.predict(X_val))
    val_proba = clf.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_proba)
    f1 = metrics.f1_score(y_val, clf.predict(X_val))
    val_evals.append(val_acc)
    aucs.append(auc)
    f1s.append(f1)
    print(train_acc, val_acc, auc, f1, '\n')
    
print(sum(val_evals) / len(val_evals))
print(sum(aucs) / len(aucs))
print(sum(f1s) / len(f1s), np.std(f1s))
    
    

(63, 13) (63,) (array([0, 1]), array([11,  5]))
0.873015873015873 0.6875 0.6727272727272727 0.5454545454545454 

(63, 13) (63,) (array([0, 1]), array([11,  5]))
0.8412698412698413 0.6875 0.6 0.28571428571428575 

(63, 13) (63,) (array([0, 1]), array([9, 7]))
0.8412698412698413 0.8125 0.7777777777777777 0.7999999999999999 

(63, 13) (63,) (array([0, 1]), array([9, 7]))
0.8412698412698413 0.8125 0.888888888888889 0.8235294117647058 

(64, 13) (64,) (array([0, 1]), array([11,  4]))
0.828125 0.6666666666666666 0.7954545454545454 0.6153846153846153 

0.7333333333333333
0.7469696969696968
0.6140165716636304 0.1954330983861492


In [4]:
from sklearn.feature_selection import RFE

# clf = SVC(kernel='sigmoid', random_state=42, probability=True, C=0.1, class_weight='balanced')
clf = LogisticRegression(solver='sag', random_state=42, class_weight='balanced')
# clf = XGBClassifier(
#         random_state = 42,
#         objective='binary:logistic',
#         n_estimators=500,
#         learning_rate=1e-3,
#         max_depth=3,
#         subsample=0.7,
#         scale_pos_weight=np.unique(y_train, return_counts=True)[1][0] / np.unique(y_train, return_counts=True)[1][1]
#     )
    

val_evals = []
aucs = []
f1s = []

for fidx in range(5):
    X_train = np.load('/work/zg78/som_folds/X_train_{}.npy'.format(fidx))
    y_train = np.load('/work/zg78/som_folds/y_train_{}.npy'.format(fidx))
    X_val = np.load('/work/zg78/som_folds/X_val_{}.npy'.format(fidx))
    y_val = np.load('/work/zg78/som_folds/y_val_{}.npy'.format(fidx))
        
    std_scaler = StandardScaler()
    std_scaler.fit(X_train)
    X_train = std_scaler.transform(X_train)
    X_val = std_scaler.transform(X_val)
    selector = RFE(clf, n_features_to_select=8, step=1)
#     clf.fit(X_train, y_train)
    selector = selector.fit(X_train, y_train)

    train_acc = metrics.accuracy_score(y_train, selector.predict(X_train))
    val_acc = metrics.accuracy_score(y_val, selector.predict(X_val))
    val_proba = selector.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, val_proba)
    f1 = metrics.f1_score(y_val, selector.predict(X_val))
    val_evals.append(val_acc)
    aucs.append(auc)
    f1s.append(f1)

    print(train_acc, val_acc, auc, f1, '\n')
    
print(sum(val_evals) / len(val_evals))
print(sum(aucs) / len(aucs))
print(sum(f1s) / len(f1s), np.std(f1s))

0.8412698412698413 0.5625 0.6727272727272727 0.4615384615384615 

0.8095238095238095 0.6875 0.6181818181818182 0.28571428571428575 

0.7936507936507936 0.8125 0.7619047619047619 0.7999999999999999 

0.7936507936507936 0.8125 0.888888888888889 0.8235294117647058 

0.8125 0.6666666666666666 0.8181818181818181 0.6153846153846153 

0.7083333333333333
0.751976911976912
0.5972333548804136 0.20401515655731906
